In [ ]:
import asyncio
from playwright.async_api import async_playwright
import aiohttp
import os
import nest_asyncio

nest_asyncio.apply()

async def download_image(session, url, filename):
    async with session.get(url) as resp:
        if resp.status == 200:
            with open(filename, 'wb') as f:
                f.write(await resp.read())

async def crawl_tour_detail(context, detail_url):
    page = await context.new_page()
    await page.goto(detail_url)
    await page.wait_for_timeout(3000)

    try:
        itinerary_buttons = await page.locator("button.accordion-button").all()
        itinerary = "\n".join([await btn.inner_text() for btn in itinerary_buttons])
        

    except:
        itinerary = "Không có dữ liệu lịch trình"

    try:
        includes = await page.locator("div.includes").inner_text()
    except:
        includes = "Không rõ dịch vụ bao gồm"

    await page.close()
    return {
        "itinerary": itinerary,
        "includes": includes
    }

async def crawl_tours():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context()
        page = await context.new_page()
        base_url = "https://hanoitourist.vn/tour-trong-nuoc"
        await page.goto(base_url)
        await page.wait_for_timeout(5000)

        current_page = 1
        max_page = 1  # Đổi số trang muốn crawl tại đây

        all_tours = []
        async with aiohttp.ClientSession() as session:
            while current_page <= max_page:
                print(f"🔎 Crawling page {current_page}...")

                tour_cards = await page.locator("div.tour-item").all()

                for i, card in enumerate(tour_cards):
                    try:
                        title = await card.locator("h3 a").inner_text()
                        price = await card.locator("div.detail-gia").inner_text()
                        location = await card.locator("div.detail-item-value >> nth=1").inner_text()
                        dates = await card.locator("div.detail-item-value >> nth=0").inner_text()
                        detail_url = await card.locator("h3 a").get_attribute("href")
                        full_detail_url = f"https://hanoitourist.vn{detail_url}" if detail_url.startswith("/") else detail_url
                        print(full_detail_url)
                        image_url = await card.locator("div.tour-img img").get_attribute("src")
                        filename = f"tour_image_{current_page}_{i+1}.jpg"

                        # if image_url:
                        #     await download_image(session, image_url, filename)

                        # Crawl chi tiết từng tour
                        detail_data = await crawl_tour_detail(context, full_detail_url)

                        tour_info = {
                            "title": title,
                            "location": location,
                            "dates": dates,
                            "price": price,
                            "image_url": image_url,
                            "detail_url": full_detail_url,
                            **detail_data
                        }

                        print(f"✅ {title}")
                        print(f"📍 {location} | 📅 {dates} | 💰 {price}")
                        print(f"📝 Lịch trình: {detail_data['itinerary'][:100]}...")
                        print("────────────")

                        all_tours.append(tour_info)
                    except Exception as e:
                        print("⚠️ Lỗi xử lý card:", e)

                # Tìm nút "Trang tiếp" để next page
                try:
                    next_btn = page.locator("a.page-link:has-text('>')")
                    if await next_btn.is_visible():
                        await next_btn.click()
                        await page.wait_for_timeout(3000)
                        current_page += 1
                    else:
                        break
                except:
                    break

        await browser.close()

        print(f"🎉 Tổng cộng thu được {len(all_tours)} tour.")
        
        # Có thể lưu `all_tours` vào file CSV hoặc JSON nếu muốn.

# Chạy
await crawl_tours()


🔎 Crawling page 1...
https://hanoitourist.vn/can-tho-soc-trang-con-dao
✅ CẦN THƠ - SÓC TRĂNG - CÔN ĐẢO
📍 Ngày đi: 19/04, 17/05, 20/06/2025... (4 ngày 3 đêm) | 📅 Hành trình: TP. Hồ Chí Minh - Cần Thơ - Côn Đảo - 4 ngày 3 đêm | 💰 8,790,000đ
📝 Lịch trình: NGÀY 01: HÀ NỘI - CẦN THƠ - SÓC TRĂNG (ĂN TỐI)
NGÀY 02: SÓC TRĂNG - CÔN ĐẢO (ĂN TRƯA, TỐI)
NGÀY 03: ...
────────────
https://hanoitourist.vn/can-tho-cao-lanh-chau-doc-giang-can-tho
✅ CẦN THƠ - CAO LÃNH - CHÂU ĐỐC - AN GIANG - CẦN THƠ
📍 Ngày đi: 19/04, 17/05, 20/06/2025... (4 ngày 3 đêm) | 📅 Hành trình: Cần Thơ - TP. Hồ Chí Minh - 4 ngày 3 đêm | 💰 8,150,000đ
📝 Lịch trình: NGÀY 1: HÀ NỘI - CẦN THƠ - CAO LÃNH (Ăn tối)
NGÀY 2: CAO LÃNH - TRI TÔN - RỪNG TRÀM - CHÂU ĐỐC (Ăn s...
────────────
https://hanoitourist.vn/vinwonder-bai-tranh-lang-chai-vinh-san-ho-dong-cuu-hang-rai-vuon-nho-ninh-thuan-thap-ba-ponagar-tam
✅ VINWONDER – BÃI TRANH – LÀNG CHÀI – VỊNH SAN HÔ ĐỒNG CỪU – HANG RÁI - VƯỜN NHO NINH THUẬN THÁP BÀ PONAGAR – TẮM BÙN I’RESORT
📍 Ngà